In [1]:
from openai import OpenAI

openai_api_key = "EMPTY"
openai_api_base = "http://localhost:2242/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

def get_llama_response(prompt, temperature = 0.1, max_tokens = 128):
    completion = client.chat.completions.create(
        model="meta-llama/Meta-Llama-3.1-8B-Instruct",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                "content": prompt
            }
        ],
        temperature = temperature,
        # max_tokens = max_tokens
    )

    return completion.choices[0].message.content

In [2]:
get_llama_response("৫ জন বাংলাদেশি বিখ্যাত কবির নাম বল?")

'বাংলাদেশের ৫ জন বিখ্যাত কবির নাম নিম্নরূপ:\n\n১. রবীন্দ্রনাথ ঠাকুর (বাঙালি)\n২. কাজী নজরুল ইসলাম\n৩. শতীর্থ (মুহম্মদ সাদিক)\n৪. কাজী জাহাঙ্গীর (কাজী ইবরাহিম)\n৫. শেখ সাইফুদ্দীন'

In [3]:
## classification

import pandas as pd
from datasets import load_dataset

# Load the dataset
dataset = load_dataset('zabir-nabil/bangla_newspaper_dataset')

# Convert to pandas dataframe
df = dataset['test_2'].to_pandas()

# Display the dataframe
df.head()

,author,category,category_bn,published_date,modification_date,tag,comment_count,title,url,content,__index_level_0__
0,খেলা ডেস্ক,sports,খেলা,"০৮ অক্টোবর ২০১৭, ১৮:২১","০৮ অক্টোবর ২০১৭, ১৯:১২",ক্রিকেট,NaN,‘৬০০’ করতে পারল না বাংলাদেশ,http://www.prothom-alo.com/sports/article/1339886,দক্ষিণ আফ্রিকার ৫৭৩/৪ রানে ইনিংস ঘোষণার জবাবে...,340598
1,-1,sports,খেলা,"২৩ এপ্রিল ২০১৬, ০১:৫০","২৩ এপ্রিল ২০১৬, ০১:৫১","খেলা,আন্তর্জাতিক ক্রিকেট",0.0,খেলার বিনিময়ে ক্ষতিপূরণ,http://www.prothom-alo.com/sports/article/838162,বেতন-ভাতা নিয়ে জটিলতায় ২০১৪ সালে ভারত সফরের ...,222811
2,-1,sports,খেলা,"২৫ নভেম্বর ২০১৫, ০১:৫৪","২৫ নভেম্বর ২০১৫, ০১:৫৫","আন্তর্জাতিক ফুটবল,খেলা",0.0,লাউড্রপের না,http://www.prothom-alo.com/sports/article/693637,২০১৬ ইউরো চ্যাম্পিয়নশিপের চূড়ান্ত পর্বে দলকে ...,177592
3,উৎপল শুভ্র,sports,খেলা,"০৩ ডিসেম্বর ২০১৮, ১৩:২০","০৪ ডিসেম্বর ২০১৮, ১২:০৬","ক্রিকেট,বাংলাদেশ ওয়েস্ট ইন্ডিজ সিরিজ,টেস্ট ক্র...",0.0,মিরপুরে এক টেস্টে দুই প্রাপ্তি,http://www.prothom-alo.com/sports/article/1568079,যখন টেস্টের তৃতীয় দিনের শেষ সেশনের খেলা চলার ক...,384712
4,-1,sports,খেলা,"১১ আগস্ট ২০১৫, ০৩:০১","১১ আগস্ট ২০১৫, ০৩:০২","আন্তর্জাতিক ফুটবল,খেলা",0.0,‘পাগলা’ সান্টো,http://www.prothom-alo.com/sports/article/599491,ফুটবলে গোল উদ্যাপন তো কতভাবেই হয়! এই যুগে ফ্রা...,159094


In [4]:
zero_shot = """আপনাকে বাংলা সংবাদপত্রের নিবন্ধগুলির একটি ডেটাসেট দেওয়া হয়েছে। আপনার কাজ হলো প্রতিটি নিবন্ধকে তার বিষয়বস্তু অনুযায়ী পূর্বনির্ধারিত ক্যাটাগরি / বিভাগগুলির একটিতে শ্রেণীবদ্ধ করা।

পূর্বনির্ধারিত ক্যাটাগরি / বিভাগগুলি হলো:
১. বাংলাদেশ
২. খেলা
৩. প্রযুক্তি
৪. বিনোদন
৫. আন্তর্জাতিক
৬. অর্থনীতি
৭. জীবনধারা
৮. মতামত
৯. শিক্ষা

"""

In [5]:
three_shot = """আপনাকে বাংলা সংবাদপত্রের নিবন্ধগুলির একটি ডেটাসেট দেওয়া হয়েছে। আপনার কাজ হলো প্রতিটি নিবন্ধকে তার বিষয়বস্তু অনুযায়ী পূর্বনির্ধারিত ক্যাটাগরি / বিভাগগুলির একটিতে শ্রেণীবদ্ধ করা।

পূর্বনির্ধারিত ক্যাটাগরি / বিভাগগুলি হলো:
১. বাংলাদেশ
২. খেলা
৩. প্রযুক্তি
৪. বিনোদন
৫. আন্তর্জাতিক
৬. অর্থনীতি
৭. জীবনধারা
৮. মতামত
৯. শিক্ষা

### উদাহরণ ১
নিবন্ধের বিষয়বস্তু:
'গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায় আজ বৃহস্পতিবার রাতের টিফিন খেয়ে একটি পোশাক কারখানার ৫০০ শ্রমিক অসুস্থ হয়ে পড়েছেন। এ ঘটনায় বিক্ষোভ করেছেন ওই কারখানার শ্রমিকেরা...'
প্রেডিক্টেড ক্যাটাগরি: 'বাংলাদেশ'

### উদাহরণ ২
নিবন্ধের বিষয়বস্তু:
'মেসি তার দুর্দান্ত পারফরম্যান্সের মাধ্যমে আবারও সবার মন জয় করে নিলেন। আজকের ম্যাচে তিনি দুইটি গোল করেছেন এবং একটি অ্যাসিস্ট দিয়েছেন।'
প্রেডিক্টেড ক্যাটাগরি: 'খেলা'

### উদাহরণ ৩
নিবন্ধের বিষয়বস্তু:
'এখন ভ্রমণের সময়। কিছু যন্ত্র ব্যাকপ্যাকে না থাকলে অনেক যন্ত্রণা! এক মোবাইল ফোনেই ক্যামেরা, কম্পাস, স্পিকার, মানচিত্র থেকে শুরু করে টর্চলাইটের সুবিধা অবশ্য মেলে।'
প্রেডিক্টেড ক্যাটাগরি: 'প্রযুক্তি'

### আপনার কাজ:

"""

In [6]:
chain_of_thought = """আপনাকে বাংলা সংবাদপত্রের নিবন্ধগুলির একটি ডেটাসেট দেওয়া হয়েছে। আপনার কাজ হলো প্রতিটি নিবন্ধকে তার বিষয়বস্তু অনুযায়ী পূর্বনির্ধারিত ক্যাটাগরি / বিভাগগুলির একটিতে শ্রেণীবদ্ধ করা।

পূর্বনির্ধারিত ক্যাটাগরি / বিভাগগুলি হলো:
১. বাংলাদেশ
২. খেলা
৩. প্রযুক্তি
৪. বিনোদন
৫. আন্তর্জাতিক
৬. অর্থনীতি
৭. জীবনধারা
৮. মতামত
৯. শিক্ষা

প্রতিটি নিবন্ধের বিষয়বস্তু বিশ্লেষণ করার সময় নিম্নলিখিত ধাপগুলি অনুসরণ করুন:

### ধাপ ১: নিবন্ধের মূল বিষয়বস্তু বোঝা
নিবন্ধের বিষয়বস্তু পড়ুন এবং এর মূল বিষয় বা ঘটনাটি বোঝার চেষ্টা করুন। এটি একটি সংবাদ ঘটনা, খেলার আপডেট, প্রযুক্তির খবর, বিনোদনের খবর, আন্তর্জাতিক খবর, অর্থনৈতিক আপডেট, জীবনধারার টিপস, মতামত বা শিক্ষার খবর হতে পারে।

### ধাপ ২: মূল থিম নির্ধারণ
নিবন্ধের প্রধান থিম বা বিষয়টি নির্ধারণ করুন। এটি নিবন্ধের শিরোনাম এবং প্রথম কয়েকটি বাক্য পড়ে সহজেই বোঝা যায়।

### ধাপ ৩: বিভাগ মিলান
নিবন্ধের প্রধান থিমের সাথে পূর্বনির্ধারিত বিভাগগুলির একটি মিলান করুন। নিশ্চিত হন যে বিভাগটি নিবন্ধের মূল বিষয়ের সাথে সুসঙ্গত।

### উদাহরণ ১
নিবন্ধের বিষয়বস্তু:
'গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায় আজ বৃহস্পতিবার রাতের টিফিন খেয়ে একটি পোশাক কারখানার ৫০০ শ্রমিক অসুস্থ হয়ে পড়েছেন। এ ঘটনায় বিক্ষোভ করেছেন ওই কারখানার শ্রমিকেরা...'

#### চিন্তাধারা:
- মূল বিষয়বস্তু: পোশাক কারখানার শ্রমিকদের অসুস্থতা এবং বিক্ষোভ
- মূল থিম: স্থানীয় বাংলাদেশি সংবাদ
- বিভাগ মিলান: বাংলাদেশ

প্রেডিক্টেড ক্যাটাগরি: 'বাংলাদেশ'

### উদাহরণ ২
নিবন্ধের বিষয়বস্তু:
'মেসি তার দুর্দান্ত পারফরম্যান্সের মাধ্যমে আবারও সবার মন জয় করে নিলেন। আজকের ম্যাচে তিনি দুইটি গোল করেছেন এবং একটি অ্যাসিস্ট দিয়েছেন।'

#### চিন্তাধারা:
- মূল বিষয়বস্তু: মেসির পারফরম্যান্স এবং খেলার ফলাফল
- মূল থিম: খেলাধুলার খবর
- বিভাগ মিলান: খেলা

প্রেডিক্টেড ক্যাটাগরি: 'খেলা'

### উদাহরণ ৩
নিবন্ধের বিষয়বস্তু:
'এখন ভ্রমণের সময়। কিছু যন্ত্র ব্যাকপ্যাকে না থাকলে অনেক যন্ত্রণা! এক মোবাইল ফোনেই ক্যামেরা, কম্পাস, স্পিকার, মানচিত্র থেকে শুরু করে টর্চলাইটের সুবিধা অবশ্য মেলে।'

#### চিন্তাধারা:
- মূল বিষয়বস্তু: মোবাইল ফোন ও গ্যাজেট
- মূল থিম: প্রযুক্তির খবর
- বিভাগ মিলান: প্রযুক্তি

প্রেডিক্টেড ক্যাটাগরি: 'প্রযুক্তি'

### আপনার কাজ:

"""

In [7]:
category_translation = {
    'bangladesh': 'বাংলাদেশ',
    'sports': 'খেলা',
    'technology': 'প্রযুক্তি',
    'entertainment': 'বিনোদন',
    'international': 'আন্তর্জাতিক',
    'economy': 'অর্থনীতি',
    'life-style': 'জীবনধারা',
    'opinion': 'মতামত',
    'education': 'শিক্ষা'
}

In [8]:
path_root = "exps/"

In [37]:
import time
from tqdm.notebook import tqdm

# Assuming df is the original DataFrame and category_translation is defined
df['predicted_category'] = None

# Try to load the progress from the Parquet file
try:
    df_progress = pd.read_parquet(path_root + 'llama_zs_classification_results.parquet')
    start_index = df_progress['predicted_category'].last_valid_index() + 1
except (FileNotFoundError, ValueError):
    df_progress = df.copy()
    start_index = 0


for i, row in tqdm(df.iloc[start_index:].iterrows(), initial=start_index, total=len(df) - start_index):
    news = row['content']
    label = row['category']
    label_bn = category_translation.get(label, label)  # Use the translated label or fallback to the original

    prompt_input = zero_shot + " নিবন্ধের বিষয়বস্তু: " + news + "\nপ্রেডিক্টেড ক্যাটাগরি:"

    try:
        df.at[i, 'predicted_category'] = get_llama_response(prompt_input)
    except Exception as e:
        print(f"Error at index {i}: {e}")
        time.sleep(5)

    if (i + 1) % 100 == 0:
        df.to_parquet(path_root + 'llama_zs_classification_results.parquet')

# Save final progress
df.to_parquet(path_root + 'llama_zs_classification_results.parquet')

  0%|          | 0/495 [00:00<?, ?it/s]

In [7]:
zs_results = pd.read_parquet(path_root + 'llama_zs_classification_results.parquet')
zs_results.head()

,author,category,category_bn,published_date,modification_date,tag,comment_count,title,url,content,__index_level_0__,predicted_category
0,খেলা ডেস্ক,sports,খেলা,"০৮ অক্টোবর ২০১৭, ১৮:২১","০৮ অক্টোবর ২০১৭, ১৯:১২",ক্রিকেট,NaN,‘৬০০’ করতে পারল না বাংলাদেশ,http://www.prothom-alo.com/sports/article/1339886,দক্ষিণ আফ্রিকার ৫৭৩/৪ রানে ইনিংস ঘোষণার জবাবে...,340598,ক্যাটাগরি ৪: খেলা
1,-1,sports,খেলা,"২৩ এপ্রিল ২০১৬, ০১:৫০","২৩ এপ্রিল ২০১৬, ০১:৫১","খেলা,আন্তর্জাতিক ক্রিকেট",0.0,খেলার বিনিময়ে ক্ষতিপূরণ,http://www.prothom-alo.com/sports/article/838162,বেতন-ভাতা নিয়ে জটিলতায় ২০১৪ সালে ভারত সফরের ...,222811,বিবেচনা করে পূর্বনির্ধারিত ক্যাটাগরি / বিভাগগু...
2,-1,sports,খেলা,"২৫ নভেম্বর ২০১৫, ০১:৫৪","২৫ নভেম্বর ২০১৫, ০১:৫৫","আন্তর্জাতিক ফুটবল,খেলা",0.0,লাউড্রপের না,http://www.prothom-alo.com/sports/article/693637,২০১৬ ইউরো চ্যাম্পিয়নশিপের চূড়ান্ত পর্বে দলকে ...,177592,"তথ্যটি বিশ্লেষণ করার পর, আমি বলতে পারি যে এই ন..."
3,উৎপল শুভ্র,sports,খেলা,"০৩ ডিসেম্বর ২০১৮, ১৩:২০","০৪ ডিসেম্বর ২০১৮, ১২:০৬","ক্রিকেট,বাংলাদেশ ওয়েস্ট ইন্ডিজ সিরিজ,টেস্ট ক্র...",0.0,মিরপুরে এক টেস্টে দুই প্রাপ্তি,http://www.prothom-alo.com/sports/article/1568079,যখন টেস্টের তৃতীয় দিনের শেষ সেশনের খেলা চলার ক...,384712,প্রেডিক্টেড ক্যাটাগরিটি হলো:\n\n9. শিক্ষা\n\nয...
4,-1,sports,খেলা,"১১ আগস্ট ২০১৫, ০৩:০১","১১ আগস্ট ২০১৫, ০৩:০২","আন্তর্জাতিক ফুটবল,খেলা",0.0,‘পাগলা’ সান্টো,http://www.prothom-alo.com/sports/article/599491,ফুটবলে গোল উদ্যাপন তো কতভাবেই হয়! এই যুগে ফ্রা...,159094,খেলা


In [14]:
print(df_progress.at[0, 'predicted_category'])

None


In [15]:
'predicted_category' in df_progress.columns

True

In [9]:
import time
from tqdm.notebook import tqdm

# Assuming df is the original DataFrame and category_translation is defined
if 'predicted_category' not in df:
    df['predicted_category'] = None

# Try to load the progress from the Parquet file
try:
    df_progress = pd.read_parquet(path_root + 'llama_ts_classification_results.parquet')
    start_index = df_progress['predicted_category'].last_valid_index() + 1
except (FileNotFoundError, ValueError):
    df_progress = df.copy()
    start_index = 0

start_index = 0 # re-populated


for i, row in tqdm(df_progress.iterrows()):
    news = row['content']
    label = row['category']
    label_bn = category_translation.get(label, label)  # Use the translated label or fallback to the original

    prompt_input = three_shot + " নিবন্ধের বিষয়বস্তু: " + news + "\nপ্রেডিক্টেড ক্যাটাগরি:"

    if df_progress.at[i, 'predicted_category'] != None:
        continue

    try:
        df_progress.at[i, 'predicted_category'] = get_llama_response(prompt_input)
    except Exception as e:
        print(f"Error at index {i}: {e}")
        time.sleep(5)

    if (i + 1) % 100 == 0:
        df_progress.to_parquet(path_root + 'llama_ts_classification_results.parquet')

# Save final progress
df_progress.to_parquet(path_root + 'llama_ts_classification_results.parquet')

0it [00:00, ?it/s]

In [12]:
pd.read_parquet(path_root + 'llama_ts_classification_results.parquet').head()

,author,category,category_bn,published_date,modification_date,tag,comment_count,title,url,content,__index_level_0__,predicted_category
0,খেলা ডেস্ক,sports,খেলা,"০৮ অক্টোবর ২০১৭, ১৮:২১","০৮ অক্টোবর ২০১৭, ১৯:১২",ক্রিকেট,NaN,‘৬০০’ করতে পারল না বাংলাদেশ,http://www.prothom-alo.com/sports/article/1339886,দক্ষিণ আফ্রিকার ৫৭৩/৪ রানে ইনিংস ঘোষণার জবাবে...,340598,'খেলা'
1,-1,sports,খেলা,"২৩ এপ্রিল ২০১৬, ০১:৫০","২৩ এপ্রিল ২০১৬, ০১:৫১","খেলা,আন্তর্জাতিক ক্রিকেট",0.0,খেলার বিনিময়ে ক্ষতিপূরণ,http://www.prothom-alo.com/sports/article/838162,বেতন-ভাতা নিয়ে জটিলতায় ২০১৪ সালে ভারত সফরের ...,222811,"আমি বলব, নিবন্ধের বিষয়বস্তু অনুযায়ী প্রেডিক্..."
2,-1,sports,খেলা,"২৫ নভেম্বর ২০১৫, ০১:৫৪","২৫ নভেম্বর ২০১৫, ০১:৫৫","আন্তর্জাতিক ফুটবল,খেলা",0.0,লাউড্রপের না,http://www.prothom-alo.com/sports/article/693637,২০১৬ ইউরো চ্যাম্পিয়নশিপের চূড়ান্ত পর্বে দলকে ...,177592,নিবন্ধের বিষয়বস্তু 'এএফপি' শব্দটি ব্যবহার করে...
3,উৎপল শুভ্র,sports,খেলা,"০৩ ডিসেম্বর ২০১৮, ১৩:২০","০৪ ডিসেম্বর ২০১৮, ১২:০৬","ক্রিকেট,বাংলাদেশ ওয়েস্ট ইন্ডিজ সিরিজ,টেস্ট ক্র...",0.0,মিরপুরে এক টেস্টে দুই প্রাপ্তি,http://www.prothom-alo.com/sports/article/1568079,যখন টেস্টের তৃতীয় দিনের শেষ সেশনের খেলা চলার ক...,384712,নিবন্ধটির বিষয়বস্তু থেকে বুঝা যায় যে এটি ক্র...
4,-1,sports,খেলা,"১১ আগস্ট ২০১৫, ০৩:০১","১১ আগস্ট ২০১৫, ০৩:০২","আন্তর্জাতিক ফুটবল,খেলা",0.0,‘পাগলা’ সান্টো,http://www.prothom-alo.com/sports/article/599491,ফুটবলে গোল উদ্যাপন তো কতভাবেই হয়! এই যুগে ফ্রা...,159094,প্রেডিক্টেড ক্যাটাগরি: 'খেলা'\n\nইউজিনিফেয়ার ...


In [13]:
pd.read_parquet(path_root + 'llama_ts_classification_results.parquet').tail()

,author,category,category_bn,published_date,modification_date,tag,comment_count,title,url,content,__index_level_0__,predicted_category
490,"ইকবাল খান, শিক্ষক",education,শিক্ষা,"০৫ মার্চ ২০১৪, ০০:৪৭","০৫ মার্চ ২০১৪, ০০:৪৮",পড়াশোনা,0.0,ই ং রে জি,http://www.prothom-alo.com/education/article/1...,"আনসিন প্যাসেজ প্রিয় শিক্ষার্থী, আজ রয়েছে একটি...",56033,"নিবন্ধের বিষয়বস্তু বিশ্লেষণ করে, আমি উপসংহারে..."
491,মোহাম্মদ হেদায়েত উল্যাহ,education,শিক্ষা,"২২ মার্চ ২০১৪, ০০:৩৩","২২ মার্চ ২০১৪, ০০:৩৫",পড়াশোনা,0.0,স মা জ বি জ্ঞা ন ১ ম প ত্র,http://www.prothom-alo.com/education/article/1...,"বহুনির্বাচনি প্রশ্নোত্তর প্রিয় শিক্ষার্থী, আজ...",60214,"নিবন্ধটির বিষয়বস্তু হল শিক্ষা বিষয়ক, এবং এই ..."
492,বাদল চৌধুরী,education,শিক্ষা,"০৮ জুলাই ২০১৩, ০০:৪৮","০৮ জুলাই ২০১৩, ০০:৫০",পড়াশোনা,0.0,জুনিয়র স্কুল সার্টিফিকেট পরীক্ষার প্রস্তুতি,http://www.prothom-alo.com/education/article/2...,"প্রিয় জেএসসি পরীক্ষার্থী, শুভেচ্ছা রইল। জাতীয...",1167,"প্রিয় জেএসসি পরীক্ষার্থী, শুভেচ্ছা রইল। জাতীয..."
493,মারুফ ইসলাম,education,শিক্ষা,"২৫ আগস্ট ২০১৩, ০০:১১","২৫ আগস্ট ২০১৩, ০০:১৬","স্বপ্ন নিয়ে,শিক্ষাঙ্গন",0.0,বিতর্ক করতে মালয়েশিয়া,http://www.prothom-alo.com/education/article/4...,সাত রাউন্ড শেষে তিনটি করে জয় নিশ্চিত করে ঢাকা...,11829,আইবিএ ওআইসি চ্যাম্পিয়ন হয়ে উঠেছে বাংলাদেশের ...
494,মো. জসিম উদ্দীন বিশ্বাস,education,শিক্ষা,"০৫ মে ২০১৪, ০১:৪৩","০৫ মে ২০১৪, ০১:৪৫",পড়াশোনা,0.0,ইংরেজি ২য় পত্র,http://www.prothom-alo.com/education/article/2...,"The Prepositionপ্রিয় শিক্ষার্থীরা, আজ ইংরেজি ২...",70843,আমি উপরোক্ত নিবন্ধটির প্রেডিক্টেড ক্যাটাগরি হব...


In [17]:
pd.read_parquet("exps/llama_cot_classification_results.parquet").tail()

,author,category,category_bn,published_date,modification_date,tag,comment_count,title,url,content,__index_level_0__,predicted_category
490,"ইকবাল খান, শিক্ষক",education,শিক্ষা,"০৫ মার্চ ২০১৪, ০০:৪৭","০৫ মার্চ ২০১৪, ০০:৪৮",পড়াশোনা,0.0,ই ং রে জি,http://www.prothom-alo.com/education/article/1...,"আনসিন প্যাসেজ প্রিয় শিক্ষার্থী, আজ রয়েছে একটি...",56033,None
491,মোহাম্মদ হেদায়েত উল্যাহ,education,শিক্ষা,"২২ মার্চ ২০১৪, ০০:৩৩","২২ মার্চ ২০১৪, ০০:৩৫",পড়াশোনা,0.0,স মা জ বি জ্ঞা ন ১ ম প ত্র,http://www.prothom-alo.com/education/article/1...,"বহুনির্বাচনি প্রশ্নোত্তর প্রিয় শিক্ষার্থী, আজ...",60214,None
492,বাদল চৌধুরী,education,শিক্ষা,"০৮ জুলাই ২০১৩, ০০:৪৮","০৮ জুলাই ২০১৩, ০০:৫০",পড়াশোনা,0.0,জুনিয়র স্কুল সার্টিফিকেট পরীক্ষার প্রস্তুতি,http://www.prothom-alo.com/education/article/2...,"প্রিয় জেএসসি পরীক্ষার্থী, শুভেচ্ছা রইল। জাতীয...",1167,None
493,মারুফ ইসলাম,education,শিক্ষা,"২৫ আগস্ট ২০১৩, ০০:১১","২৫ আগস্ট ২০১৩, ০০:১৬","স্বপ্ন নিয়ে,শিক্ষাঙ্গন",0.0,বিতর্ক করতে মালয়েশিয়া,http://www.prothom-alo.com/education/article/4...,সাত রাউন্ড শেষে তিনটি করে জয় নিশ্চিত করে ঢাকা...,11829,None
494,মো. জসিম উদ্দীন বিশ্বাস,education,শিক্ষা,"০৫ মে ২০১৪, ০১:৪৩","০৫ মে ২০১৪, ০১:৪৫",পড়াশোনা,0.0,ইংরেজি ২য় পত্র,http://www.prothom-alo.com/education/article/2...,"The Prepositionপ্রিয় শিক্ষার্থীরা, আজ ইংরেজি ২...",70843,None


In [19]:
import time
from tqdm.notebook import tqdm

# Assuming df is the original DataFrame and category_translation is defined
if 'predicted_category' not in df:
    df['predicted_category'] = None

# Try to load the progress from the Parquet file
try:
    df_progress = pd.read_parquet(path_root + 'llama_cot_classification_results.parquet')
    start_index = df_progress['predicted_category'].last_valid_index() + 1
except (FileNotFoundError, ValueError):
    df_progress = df.copy()
    start_index = 0


for i, row in tqdm(df.iloc[start_index:].iterrows(), initial=start_index, total=len(df) - start_index):
    news = row['content']
    label = row['category']
    label_bn = category_translation.get(label, label)  # Use the translated label or fallback to the original

    prompt_input = chain_of_thought + " নিবন্ধের বিষয়বস্তু: " + news

    try:
        df.at[i, 'predicted_category'] = get_llama_response(prompt_input)
    except Exception as e:
        print(f"Error at index {i}: {e}")
        time.sleep(5)

    if (i + 1) % 25 == 0:
        df.to_parquet(path_root + 'llama_cot_classification_results.parquet')

# Save final progress
df.to_parquet(path_root + 'llama_cot_classification_results.parquet')

 11%|#1        | 50/445 [00:00<?, ?it/s]

In [22]:
def clean_predicted_category(text, category_translation):
    # Check if the text is None
    if text is None:
        return ""

    # Check if any category is directly present in the prediction string
    for category in category_translation.values():
        if category in text:
            return category

    # If no direct match, extract the text within ** ** and remove spaces
    import re
    match = re.search(r'\*\*([^*]*)\*\*', text)
    if match:
        cleaned_text = match.group(1).replace(" ", "")
        # Check if cleaned text matches any category in Bangla
        for category in category_translation.values():
            if cleaned_text == category:
                return category

    # If no match found, return None
    return None

In [20]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss
from sklearn.preprocessing import LabelBinarizer


def evaluate_classification(df):
    # Map English category to Bangla
    df['category_bn'] = df['category'].map(category_translation)

    # Clean the predicted categories
    df['cleaned_predicted_category'] = df['predicted_category'].apply(clean_predicted_category, args=(category_translation,))

    # Replace None values with an incorrect label to be treated as wrong predictions
    df['cleaned_predicted_category'] = df['cleaned_predicted_category'].fillna('wrong_prediction')

    # Calculate metrics
    y_true = df['category_bn']
    y_pred = df['cleaned_predicted_category']

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='macro', zero_division=0)
    recall = recall_score(y_true, y_pred, average='macro', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)

    # For AUC-ROC and Log Loss, we need to binarize the labels
    lb = LabelBinarizer()
    y_true_binarized = lb.fit_transform(y_true)
    y_pred_binarized = lb.transform(y_pred)

    # Not useful in this setup
    auc_roc = roc_auc_score(y_true_binarized, y_pred_binarized, average='macro', multi_class='ovr')
    logloss = log_loss(y_true_binarized, y_pred_binarized)

    metrics = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'auc_roc': auc_roc,
        'log_loss': logloss
    }

    return metrics

In [24]:
zs_df = pd.read_parquet(path_root + "llama_zs_classification_results.parquet")
metrics = evaluate_classification(zs_df)
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

accuracy: 0.4606
precision: 0.5353
recall: 0.4145
f1_score: 0.4242
auc_roc: 0.6985
log_loss: 19.4417


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:2956: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(


In [25]:
ts_df = pd.read_parquet(path_root + "llama_ts_classification_results.parquet")
metrics = evaluate_classification(ts_df)
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

accuracy: 0.4667
precision: 0.5723
recall: 0.4200
f1_score: 0.4351
auc_roc: 0.7016
log_loss: 19.2233


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:2956: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(


In [26]:
cot_df = pd.read_parquet(path_root + "llama_cot_classification_results.parquet")
metrics = evaluate_classification(cot_df)
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

accuracy: 0.3798
precision: 0.4339
recall: 0.3107
f1_score: 0.3243
auc_roc: 0.6593
log_loss: 22.3543


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:2956: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(
